In [ ]:
import sys
sys.path.append('../../preprocess_assets/')

In [ ]:
# Main libraries 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

# Our files
from data_shuffling_split import *
from features_extraction import *
from ara_vec_preprocess_configs import *
from ml_modeling import *

In [ ]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

In [ ]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

In [ ]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

In [ ]:
# Get how many words inside each text after tokenization
num_of_words_in_each_text = [len(text) for text in fullgram_x_train_text_tokenized]
max_len = max(num_of_words_in_each_text)
print("The max length is: ", max_len)
num_of_words_in_each_text[:10]

In [ ]:
# count how many times these value repeated and sort them
new_dicts = get_keys_that_val_gr_than_num(num_of_words_in_each_text, 1000)
keys = list(new_dicts.keys())
values = list(new_dicts.values())
plt.style.use('dark_background')
fig = plt.gcf()
fig.set_size_inches(18.5, 6)
plt.bar(range(len(new_dicts)), values, tick_label=keys)
plt.show()

In [ ]:
word2vec_model = load_word2vec_model("../word2vec_models/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
number_of_features = 300
max_len_str = 128
word2vec_path = "rezk/"
model_path_to_save = "../ml_models_saved/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word2vec_model, fullgram_x_val_text_tokenized, max_len_str)

print(X_train_embed_matrix.shape)
print("="*50)
print(X_val_embed_matrix.shape)
print("="*50)

# Train Logistic Regression

In [ ]:
model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Train SVC

In [ ]:
model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Hard Voting

In [ ]:
model = VotingClassifier(estimators, voting="hard")
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 

In [ ]:
model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

#  Gradient Boosting

In [ ]:
model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Load best model & predict test set

In [ ]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

In [ ]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [ ]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

In [ ]:
X_test_embed_matrix = text_to_matrix_using_word2vec(word2vec_model, fullgram_X_test_text_tokenized, max_len_str)

print(X_test_embed_matrix.shape)
print("="*50)

In [ ]:
model = pickle_load_model("ml_models_saved/rezk/" + )
f1_score_result(model, X_test_embed_matrix, y_test):

# Qualitative Evaluation

In [4]:
!pip3 install manimce

  Using cached manimce-0.1.1.post2-py3-none-any.whl (249 kB)
  Using cached cairocffi-1.3.0-py3-none-any.whl
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached pangocairocffi-0.4.0-py3-none-any.whl
  Using cached progressbar-2.5-py3-none-any.whl
  Using cached rich-6.2.0-py3-none-any.whl (150 kB)
  Using cached pangocffi-0.8.0-py3-none-any.whl
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached pycairo-1.21.0.tar.gz (340 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pycairo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_6

In [2]:
!pip3 install mplcairo

  Using cached mplcairo-0.4-cp39-cp39-manylinux1_x86_64.whl (858 kB)
  Using cached pycairo-1.21.0.tar.gz (340 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pycairo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/cairo
      copying cairo/__init__.py -> build/lib.linux-x86_64-cpython-39/cairo
      copying cairo/__init__.pyi -> build/lib.linux-x86_64-cpython-39/cairo
      copying cairo/py.typed -> build/lib.linux-x86_64-cpython-39/cairo
      running build_ext
      Package cairo was not found in the pkg-config search path.
      Perhaps you should add the directory containing `cairo.pc'
      to the PKG_CON

In [1]:
import matplotlib, mplcairo
print('Default backend: ' + matplotlib.get_backend()) 
matplotlib.use("module://mplcairo.macosx")
print('Backend is now ' + matplotlib.get_backend())

ModuleNotFoundError: No module named 'mplcairo'